In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [11]:
col_names = ['fLength', 'fWidth', 'fSize', 'fConc', 'fConc1',
             'fAsym', 'fM3Long', 'fM3Trans', 'fAlpha', 'fDist', 'class',]
df = pd.read_csv('magic04.data', names=col_names)
df.head()
df.shape

,fLength,fWidth,fSize,fConc,fConc1,fAsym,fM3Long,fM3Trans,fAlpha,fDist,class
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,g
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,g
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,g
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,g
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,g


(19020, 11)

In [12]:
df['class'] = (df['class'] == 'g').astype(int)
df['class'].unique()

array([1, 0])

In [13]:
train, valid, test = np.split(df.sample(frac=1, random_state=1), [int(len(df)*.6), int(len(df)*.8)])

train.shape, valid.shape, test.shape

c:\Users\k26ra\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


((11412, 11), (3804, 11), (3804, 11))

In [14]:
def scale_oversample(df, oversample=False):
  from sklearn.preprocessing import StandardScaler
  from imblearn.over_sampling import RandomOverSampler

  X = df[df.columns[:-1]].values
  y = df[df.columns[-1]].values

  scaler = StandardScaler()
  X = scaler.fit_transform(X)

  if oversample:
    ros = RandomOverSampler()
    X,   y = ros.fit_resample(X, y)

  data = np.hstack((X, y.reshape((-1, 1))))
  return data, X, y


train_scaled, train_X, train_y = scale_oversample(train, oversample=True)
valid_scaled, valid_X, valid_y = scale_oversample(valid)
test_scaled, test_X, test_y = scale_oversample(test)

train_scaled.shape, train_X.shape, train_y.shape
valid_scaled.shape, valid_X.shape, valid_y.shape
test_scaled.shape, test_X.shape, test_y.shape

((14694, 11), (14694, 10), (14694,))

((3804, 11), (3804, 10), (3804,))

((3804, 11), (3804, 10), (3804,))

In [15]:
train.shape
train_scaled.shape

sum(train['class'] == 1)  # gamma
sum(train['class'] == 0)

sum(train_y == 1)  # gamma
sum(train_y == 0)

(11412, 11)

(14694, 11)

7347

4065

np.int64(7347)

np.int64(7347)

# kNN

In [16]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

In [17]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(train_X, train_y)

KNeighborsClassifier()

In [18]:
y_pred = knn.predict(test_X)
y_pred  # predicted
test_y  # actual

print(classification_report(test_y, y_pred))

array([0, 1, 0, ..., 1, 1, 0])

array([0, 0, 0, ..., 1, 1, 1])

              precision    recall  f1-score   support

           0       0.74      0.74      0.74      1294
           1       0.87      0.87      0.87      2510

    accuracy                           0.82      3804
   macro avg       0.80      0.80      0.80      3804
weighted avg       0.82      0.82      0.82      3804

